# Burrows-Wheeler Transform 

1. BWT 
2. deBWT
3. BWT_compress
4. BWT_countlist

In [2]:
import sys
import string
import random

In [11]:
# string to BWT
# the input can't be longer than 1000000
def BWT(s='BANANA'):
#     MAX_LEN = 1000000
#     if len(s) > MAX_LEN:
#         raise Exception(
#             'string too long, abandom executing BWT. Length: ' + len(s))

    s = '$' + s
    matrix = [s]
    for i in range(len(s)-1):
        s = s[1:] + s[0]
        matrix.append(s)
    matrix = sorted(matrix)

#     for m in matrix:
#         print(m)

    bwt_L = ''.join([m[-1] for m in matrix])
    bwt_F = ''.join([m[0] for m in matrix])

    # get index dict
    index_F = {}
    curr_letter = ''
    count = 0
    begin_idx = 0
    for idx in range(len(bwt_F)):
        letter = bwt_F[idx]
        if curr_letter != letter:
            # new letter, flush
            if count != 0:
                index_F[curr_letter] = (begin_idx, count)
                count = 0
                begin_idx = idx
            curr_letter = letter
        count += 1
    # flush the last one
    if count != 0:
        index_F[curr_letter] = (begin_idx, count)

    # get rank_L
    d = {}
    rank_L = []
    for letter in bwt_L:
        if letter not in d:
            d[letter] = 0
            rank_L.append(0)
        else:  # not first time
            d[letter] += 1
            rank_L.append(d[letter])

    return bwt_F, bwt_L, index_F, rank_L

In [12]:
print(BWT('abaaba'))

('$aaaabb', 'abba$aa', {'$': (0, 1), 'a': (1, 4), 'b': (5, 2)}, [0, 0, 1, 1, 0, 2, 3])


In [13]:
BWT('tomorrow and tomorrow and tomorrow')[1]

'wwddw  nnoooaatttmmmrrrrrrooo$  ooo'

In [21]:
# decompress a bwt string using LF mapping
def BWT_decomp(bwt_F = '$aaaabb', bwt_L = 'abba$aa'):
    # re-rank
    d = {}
    rank_L = []
    for letter in bwt_L:
        if letter not in d:
            d[letter] = 0
            rank_L.append(0)
        else: # not first time
            d[letter] += 1
            rank_L.append(d[letter])
    d = {}
    rank_F = []
    for letter in bwt_F:
        if letter not in d:
            d[letter] = 0
            rank_F.append(0)
        else: # not first time
            d[letter] += 1
            rank_F.append(d[letter])
    # find decomp
    decomp_str = ['$']
    idx_curr = bwt_F.index('$')
#     idx_L = bwt_L.index('$')
    letter_curr = '$'
    rank_curr = rank_F[idx_curr]
    for i in range(len(bwt_L)-1):
        # find the next i
        for idx in range(len(bwt_L)):
            if bwt_L[idx] == letter_curr and rank_L[idx] == rank_curr:
                idx_curr = idx
                break
        letter_curr = bwt_F[idx_curr]
        rank_curr = rank_F[idx_curr]
        decomp_str.append(letter_curr)
    return ''.join(decomp_str)

In [45]:
# test code
for i in range(1000):
    l = 100
    letters = string.ascii_lowercase
    s = ''.join(random.choice(letters) for i in range(l)) 
    tmp_F, tmp_L,_,_ = BWT(s)
    if BWT_decomp(tmp_F, tmp_L )[1:] != s:
        raise Exception(s)



tmp_F, tmp_L,_,_ = BWT("tomorrow and tomorrow and tomorrow")
# print(tmp_F)
# print(tmp_L)

## Strict Search in BWT

In [72]:
def strict_search_BWT(target, BWT_F, BWT_L, index_F, rank_L):
    """
    This algorithm only gives the beginning index at BWT_L instead of the original string.
    If we want to know the exact idx at original string, it's O(len(reference)) and use another algorithm.
    """
    # check if target has unseen letter:
    for i in range(len(target)):
        c = target[i]
        if c not in index_F:
            print("target has unseen letter for BWT string. Unseen letter: " + str(c) + " at Index: " + str(i))
            return 0
    offset = index_F[target[-1]][0]
    # index at BWT_F that match
    possible_idx = [i+offset for i in range(index_F[target[-1]][1])]
    # get all that matches target, idx is in BWT_F
    for i in range(len(target)-1):
        idx = len(target) - 1 - i
        letter_curr = target[idx]
        letter_prev = target[idx-1]
        possible_update = []
        # get ones still satisfy
        for j in possible_idx:
            # not satisfy
            if letter_curr != BWT_F[j] or letter_prev != BWT_L[j]:
                continue
            else:  # satisty
                possible_update.append(rank_L[j])
        # update possible_idx
        offset = index_F[letter_prev][0]
        possible_idx = [offset+t for t in possible_update]
    return possible_idx
    
    

In [73]:
print(BWT('abaaba'))

('$aaaabb', 'abba$aa', {'$': (0, 1), 'a': (1, 4), 'b': (5, 2)}, [0, 0, 1, 1, 0, 2, 3])


In [74]:
strict_search_BWT('abaas', '$aaaabb', 'abba$aa', {'$': (0, 1), 'a': (1, 4), 'b': (5, 2)}, [0, 0, 1, 1, 0, 2, 3])

target has unseen letter for BWT string. Unseen letter: s at Index: 4


0

In [16]:
s

97

## Time complexity test
### 1. BWT Test

In [27]:
import time  # 引入time模块

    
seq = "BANANA"
start_time = time.time()
BWT(seq)
time_used = format(time.time() - start_time, '.3e')
print("Time used: "+ time_used+" seconds")

Time used: 9.084e-05 seconds


In [39]:
import random

def rand_seq(l):
    alphabet = ['A','T','C','G']
    return ''.join(random.choices(alphabet, k=l))
rand_seq(10)

def time_test_BWT(l):
    seq = rand_seq(l)
    start_time = time.time()
    BWT(seq)
    time_used = format(time.time() - start_time, '.3e')
    print("Time used: "+ time_used+" seconds")

In [37]:
x = [10**i for i in range(6)]
x

[1, 10, 100, 1000, 10000, 100000]

In [44]:
time_test_BWT(100000)

TypeError: can only concatenate str (not "int") to str